In [4]:
#make this relevant, make this interesting to others, elevate this from being just a plain exercise in nerdery

#CHECK: so I'm double or triple counting nda and upa seats for every row, i've tried correcting for that, does  in any way?
#CHECK: does the fact that some seats are nominated and the fact that some of them have party affiliations screw up anything?
#CHECK: does the fact that some seats are marked VACANT, does that screw up anything here?
#CHECK: introduce conditional for when state is Delhi or Punjab, separate elections issue
#TO-DO: also prepare the data for the dotted line that shows what will happen if things remain the same
#TO-DO: there are a lot of variables being repeated, make sure you make the variable names slightly different,to be on safe side 

import csv
import datetime
import pandas as pd
import collections
import math

df_assembly_dates = pd.read_csv('assembly_end_dates.csv')
    #this dataframe has the dates for when the terms for each state assembly will begin and end
      #wasn't aware of this earlier, but it seems the Jammu & Kashmir assembly has a six-year term

df_assembly_dates['term_ends'] = pd.to_datetime(df_assembly_dates['term_ends'], format="%d-%m-%Y").dt.date
    #got code above from https://stackoverflow.com/questions/17134716/convert-dataframe-column-type-from-string-to-datetime
    #and from https://stackoverflow.com/a/34277514
        
df_assembly_data = pd.read_csv('assembly_alliances_strength.csv')
    #this has details for how many seats each party holds in each state assembly, and which alliance the party is loyal to

df_rs_data = pd.read_csv('rajya_sabha_alliances_strength.csv')
    #this has info for each rajya sabha seat, who holds it, when does their term end, which alliance do they belong to?

df_rs_data['retirement_date'] = pd.to_datetime(df_rs_data['retirement_date'], format="%d-%m-%Y").dt.date

#print df_rs_data['retirement_date'].dtype


headings = ['date', 'NDA', 'UPA', 'Others']

with open("alliances_over_time_realistic.csv", "wb") as filex:   #after this change mode to ab for append
    wr = csv.writer(filex, delimiter = ',' , quotechar = '"' )
    wr.writerow(headings)
    #seems you have to use the write mode as 'wb' instead of 'w', it's some Windows quirk - https://stackoverflow.com/a/3191811

#creating a range of dates from Aug 19, 2017 to Aug 19, 2023

#got the code below from https://stackoverflow.com/a/24637447

start = datetime.datetime.strptime("19-08-2017", "%d-%m-%Y")
end = datetime.datetime.strptime("01-07-2024", "%d-%m-%Y")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for this_date_raw in date_generated:
    this_date = this_date_raw.date()
    #print type(this_date)
    print this_date
    NDA = 0
    UPA = 0
    
    #Initialise variables NDA = 0, UPA = 0

    #for each row in the rs_data csv, ie. for each of the 245 seats in the Rajya Sabha
    
    for index, row in df_rs_data.iterrows():

        #rs_seat_retirement_date_raw = row['retirement_date']
        #rs_seat_retirement_date = datetime.datetime.strptime(rs_seat_retirement_date_raw, "%d-%m-%Y")
        rs_seat_retirement_date = row['retirement_date']
        
        #print type(rs_seat_retirement_date)
        
        #the line above is is about making python recognise the object as a date and not as a text string
        #the stackoverflow link below clarifies why it's datetime.datetime and not just datetime: 
        #https://stackoverflow.com/a/12070240
                
        alliance = row['alliance']
        
        state = row['state']
        
        rs_mp_name = row['name'] #note this could be marked 'VACANT' too, have contingency plans for that
        
        #assembly_end_date_raw = str(df_assembly_dates[df_assembly_dates['state'] == state]['term_ends']).strip('\n')
        
        if state != 'Nominated':
            #assembly_end_date_raw = df_assembly_dates[df_assembly_dates['state'] == state]['term_ends'].values[0]
            #assembly_end_date = datetime.datetime.strptime(assembly_end_date_raw, "%d-%m-%Y")
            #code above from https://stackoverflow.com/a/36684095
            assembly_end_date = df_assembly_dates[df_assembly_dates['state'] == state]['term_ends'].values[0]
        
            if this_date <= rs_seat_retirement_date: 
                if rs_mp_name != 'VACANT':

                    if alliance == 'NDA':
                        NDA += 1
                    elif alliance == 'UPA':
                        UPA += 1

                else:  # ie. if seat is marked 'vacant'

                    if this_date <= assembly_end_date:

                        diff_bw_this_date_retirement_date = rs_seat_retirement_date - this_date

                        if diff_bw_this_date_retirement_date.days > 365:

                            #now if a seat is VACANT, 
                            #want to calculate what the retirement date for the next group of vacancies from the state will be

                            #next_this_state_retirement_date = df_rs_data[(df_rs_data['state'] == state) &\
                                               #(df_rs_data['retirement_date'] < rs_seat_retirement_date)].min()["retirement_date"]
                            #had used 'not equal to' operator earlier, using less than '<' rs_seat_retirement_date now 

                            #next_any_state_retirement_date = df_rs_data[(df_rs_data['state'] != state) &\
                                               #(df_rs_data['retirement_date'] < rs_seat_retirement_date)].min()["retirement_date"]

                            next_any_state_retirement_date = df_rs_data[(df_rs_data['retirement_date'] >= this_date) &\
                                               (df_rs_data['retirement_date'] < rs_seat_retirement_date)].min()["retirement_date"]

                            #date_difference_this_state = rs_seat_retirement_date - next_this_state_retirement_date

                            #what the code above does is tell python to find the retirement date for the next group of seats

                            date_difference_any_state = rs_seat_retirement_date - next_any_state_retirement_date

                            #am assuming that if there's less than a year for the term for the seat to end, the Election 
                                #commission would rather leave the seat vacant till the term is up and not bother with holding 
                                #two elections for the same seat within a year
                                #but that if there's more than a year left, the commission might organise the election

                            #am assuming that the election commission will hold the by-poll for 
                            #a vacant RS seat when there are elections for other RS seats going on from the same state
                            #they won't be grouped together though
                            #the elections could be held sooner too, but it won't be held later than that
                            #this is all about calculating the most recent date by which these vacancies are guaranteed 
                            #to be filled

                            #I was going to have the program have elections held whenever a RS election is being held
                            #next for any state. But the EC had a chance to fill the vacancies for Telangana and 
                            #Madhya Pradesh --the MPs died --when it was holding elections for these Gujarat and WB seats
                            #but it didn't. So am just using the dates when the next elections for seats from that state
                            #is being done ******CHECK LANGUAGE OF COMMENTS, SHOULDNT" BE CONFUSING *********

                            if date_difference_any_state.days > 365:

                                if this_date > next_any_state_retirement_date:

                                    seats_vacant_now_state2 = len(df_rs_data[(df_rs_data['state'] == state) &\
                                                      (df_rs_data['retirement_date'] == rs_seat_retirement_date) &\
                                                      (rs_mp_name == 'VACANT')])

                                    assembly_seats_total_v2 = df_assembly_data[df_assembly_data['state'] == state].sum()["seats"]

                                    droop_quota_v2 = math.floor((assembly_seats_total_v2 * 100)/(seats_vacant_now_state2 + 1)) + 1

                                    #now i've seen the droop quota calculated in some places as 
                                        #math.floor((assembly_seats_total_v2)/(seats_vacant_now_state2 + 1)) + 1
                                        #I will do it the way the election commission has done it in their 
                                        #handbook for election officials, the link is http://bit.ly/2vy4cyX

                                    nda_strength_v2 = df_assembly_data[(df_assembly_data['state'] == state) &\
                                                   (df_assembly_data['alliance'] == "NDA")].sum()["seats"]
                                    #code above from https://stackoverflow.com/a/20995313

                                    upa_strength_v2 = df_assembly_data[(df_assembly_data['state'] == state) &\
                                                   (df_assembly_data['alliance'] == "UPA")].sum()["seats"]

                                    #below I'm calculating the number of seats any party will get in an RS election for a 
                                        #specific number of vacancies

                                    nda_seats_v2 = math.floor((nda_strength_v2 * 100)/droop_quota_v2)

                                    upa_seats_v2 = math.floor((upa_strength_v2 * 100)/droop_quota_v2)

                                    #I'll be rounding down the result, so am using int function here, 
                                        #got idea from https://stackoverflow.com/a/17142719

                                    NDA += nda_seats_v2/seats_vacant_now_state2

                                    UPA += upa_seats_v2/seats_vacant_now_state2
            
            else: # ie. this_date > rs_seat_retirement_date

                if this_date <= assembly_end_date:

                    if state not in ['Delhi', 'Punjab']:

                    #no else statement if this_date > assembly_end_date, it becomes part of OTHERS grouping
                    #we're just calculating min. guaranteed UPA and NDA seats here

                        #find number of seats from the state which share the same retirement date

                        vacancies_number_v3 = len(df_rs_data[(df_rs_data['state'] == state) &\
                                                          (df_rs_data['retirement_date'] == rs_seat_retirement_date)])
                        #code above from https://stackoverflow.com/a/17322585 

                        #calculate droop quota , crudely put, it's the no. of seats a party or alliance should
                            #have in a state legislature to guarantee a minimum of one Rajya Sabha seat from the RS vacancies
                            #being contested

                        #you can read more about droop_quota at ********* GIVE LINK HERE **********

                        assembly_seats_total_v3 = df_assembly_data[df_assembly_data['state'] == state].sum()["seats"]

                        droop_quota_v3 = math.floor((assembly_seats_total_v3 * 100)/(vacancies_number_v3 + 1)) + 1

                        nda_strength_v3 = df_assembly_data[(df_assembly_data['state'] == state) &\
                                       (df_assembly_data['alliance'] == "NDA")].sum()["seats"]
                        #code above from https://stackoverflow.com/a/20995313

                        upa_strength_v3 = df_assembly_data[(df_assembly_data['state'] == state) &\
                                       (df_assembly_data['alliance'] == "UPA")].sum()["seats"]

                        #below I'm calculating the number of seats any party will get in an RS election for a 
                            #specific number of vacancies

                        nda_seats_v3 = math.floor((nda_strength_v3 * 100)/droop_quota_v3)

                        upa_seats_v3 = math.floor((upa_strength_v3 * 100)/droop_quota_v3)

                        #I'll be rounding down the result, so am using int function here, 
                            #got idea from https://stackoverflow.com/a/17142719

                        NDA += nda_seats_v3/vacancies_number_v3

                        UPA += upa_seats_v3/vacancies_number_v3

                        #realised later that I'm double or triple counting the no. of seats an alliance would get
                        #basically by dividing it by the number of vacancies, I'm ensuring that when everything is summed up later
                        #you get back the actual no. of minimum guaranteed nda_seats and upa_seats

                    elif state == 'Delhi':

                        vacancies_number_d = 1 

                        #so delhi has 3 RS seats, all with terms ending on the same day
                        #but the vacancies will not be counted together
                        #there will be separate elections for each of the three seats
                        #below is the link to the Election commission notification explaining why:
                            #http://eci.nic.in/eci_main1/current/PN19122011.pdf 
                            #and an official press release about it: http://pib.nic.in/newsite/PrintRelease.aspx?relid=79013

                        assembly_seats_total_d = df_assembly_data[df_assembly_data['state'] == 'Delhi'].sum()["seats"]

                        droop_quota_d = math.floor((assembly_seats_total_d * 100)/(vacancies_number_d + 1)) + 1

                        nda_strength_d = df_assembly_data[(df_assembly_data['state'] == 'Delhi') &\
                                       (df_assembly_data['alliance'] == "NDA")].sum()["seats"]
                        #code above from https://stackoverflow.com/a/20995313

                        upa_strength_d = df_assembly_data[(df_assembly_data['state'] == 'Delhi') &\
                                       (df_assembly_data['alliance'] == "UPA")].sum()["seats"]

                        #below I'm calculating the number of seats any party will get in an RS election for a 
                            #specific number of vacancies

                        nda_seats_d = math.floor((nda_strength_d * 100)/droop_quota_d)

                        upa_seats_d = math.floor((upa_strength_d * 100)/droop_quota_d)

                        #I'll be rounding down the result, so am using int function here, 
                            #got idea from https://stackoverflow.com/a/17142719

                        NDA += nda_seats_d #Do I need to divide by vacancies_number_d here? No

                        UPA += upa_seats_d 

                    elif state == 'Punjab':

                        punjab_dates_list = df_rs_data[df_rs_data['state'] == 'Punjab']['retirement_date'].tolist()

                        #the line above gives us a list of retirement dates for RS MPs from Punjab

                        punjab_dates_freq = dict(collections.Counter(punjab_dates_list))

                        #the line above gives us a frequency table of those dates

                        #got the code above from https://stackoverflow.com/a/23240989

                        #so there are five seats (out of 7) with the same retirement date and another group of 2

                        #from that first group of five, there are two separate elections, one for 3 seats and one for 2 seats

                        #below is the link to the Election commission notification:
                        #http://eci.nic.in/eci_main1/current/PN15_24022016.pdf


                        #from the frequency table, we can identify those 5 seats with the same retirement date


                        if punjab_dates_freq[rs_seat_retirement_date] == 5: #if the retirement date is the one for the five seats

                            vacancies_number_p_g2 = 2 #g2 for group of two

                            vacancies_number_p_g3 = 3 #g3 for group of three

                            assembly_seats_total_p = df_assembly_data[df_assembly_data['state'] == 'Punjab'].sum()["seats"]                    

                            droop_quota_p_g2 = math.floor((assembly_seats_total_p * 100)/(vacancies_number_p_g2 + 1)) + 1

                            droop_quota_p_g3 = math.floor((assembly_seats_total_p * 100)/(vacancies_number_p_g3 + 1)) + 1

                            nda_strength_p = df_assembly_data[(df_assembly_data['state'] == 'Punjab') &\
                                           (df_assembly_data['alliance'] == "NDA")].sum()["seats"]
                            #code above from https://stackoverflow.com/a/20995313

                            upa_strength_p = df_assembly_data[(df_assembly_data['state'] == 'Punjab') &\
                                           (df_assembly_data['alliance'] == "UPA")].sum()["seats"]

                            nda_seats_p_g2 = math.floor((nda_strength_p * 100)/droop_quota_p_g2)

                            upa_seats_p_g2 = math.floor((upa_strength_p * 100)/droop_quota_p_g2)

                            nda_seats_p_g3 = math.floor((nda_strength_p * 100)/droop_quota_p_g3)

                            upa_seats_p_g3 = math.floor((upa_strength_p * 100)/droop_quota_p_g3)

                            #I'll be rounding down the result, so am using int function here, 
                                #got idea from https://stackoverflow.com/a/17142719

                            NDA += (nda_seats_p_g2 + nda_seats_p_g3)/5  #Is dividing by 5 the right thing here?
                                                                        #guess so, since the number of seats NDA & UPA seats
                                                                        #will be counted five times

                            UPA += (upa_seats_p_g2 + upa_seats_p_g3)/5 


                        else: #if it's the group of two RS seats from Punjab that share the same retirement date

                            vacancies_number_p2 = len(df_rs_data[(df_rs_data['state'] == 'Punjab') &\
                                          (df_rs_data['retirement_date'] == rs_seat_retirement_date)])

                            assembly_seats_total_p2 = df_assembly_data[df_assembly_data['state'] == 'Punjab'].sum()["seats"]

                            droop_quota_p2 = math.floor((assembly_seats_total_p2 * 100)/(vacancies_number_p2 + 1)) + 1

                            nda_strength_p2 = df_assembly_data[(df_assembly_data['state'] == 'Punjab') &\
                                           (df_assembly_data['alliance'] == "NDA")].sum()["seats"]
                            #code above from https://stackoverflow.com/a/20995313

                            upa_strength_p2 = df_assembly_data[(df_assembly_data['state'] == 'Punjab') &\
                                           (df_assembly_data['alliance'] == "UPA")].sum()["seats"]

                            #below I'm calculating the number of seats any party will get in an RS election for a 
                                #specific number of vacancies

                            nda_seats_p2 = math.floor((nda_strength_p2 * 100)/droop_quota_p2)

                            upa_seats_p2 = math.floor((upa_strength_p2 * 100)/droop_quota_p2)

                            #I'll be rounding down the result, so am using int function here, 
                                #got idea from https://stackoverflow.com/a/17142719

                            NDA += nda_seats_p2/vacancies_number_p2 #Do I need to divide by vacancies_number_p here?

                            UPA += upa_seats_p2/vacancies_number_p2 
        
        else: #if it is a nominated seat, the 'state' is marked 'nominated' in the data
            if this_date <= rs_seat_retirement_date:
                if alliance == 'NDA':
                    NDA += 1
                elif alliance == 'UPA':
                    UPA += 1
                #and if there is no alliance affiliation recorded against that MP, do nothing
                #and if the date is after the retirement date, we can't say anything anyway
                #note that here I know there are no vacancies among nominated seats, so I didn't provide an if/else for that

    OTHERS = 245 - (NDA + UPA)
    
    #ORIGINAL PLAN -- Write row 'date, NDA, UPA, Others, vacancies_unfilled_yet(aka vacancies_unelected),
    #vacancies_not_calculable. the larger category 'Others' will essentially be (total seats in RS - [NDA + UPA])
    
    csv_row = list()
    csv_row.append(this_date)
    csv_row.append(NDA)
    csv_row.append(UPA)
    csv_row.append(OTHERS)
     
    with open("alliances_over_time_realistic.csv", "ab") as filez:
        wrz = csv.writer(filez, delimiter = ',' , quotechar = '"' )
        wrz.writerow(csv_row)

filez.close()

print "program done"

2017-08-19
2017-08-20
2017-08-21
2017-08-22
2017-08-23
2017-08-24
2017-08-25
2017-08-26
2017-08-27
2017-08-28
2017-08-29
2017-08-30
2017-08-31
2017-09-01
2017-09-02
2017-09-03
2017-09-04
2017-09-05
2017-09-06
2017-09-07
2017-09-08
2017-09-09
2017-09-10
2017-09-11
2017-09-12
2017-09-13
2017-09-14
2017-09-15
2017-09-16
2017-09-17
2017-09-18
2017-09-19
2017-09-20
2017-09-21
2017-09-22
2017-09-23
2017-09-24
2017-09-25
2017-09-26
2017-09-27
2017-09-28
2017-09-29
2017-09-30
2017-10-01
2017-10-02
2017-10-03
2017-10-04
2017-10-05
2017-10-06
2017-10-07
2017-10-08
2017-10-09
2017-10-10
2017-10-11
2017-10-12
2017-10-13
2017-10-14
2017-10-15
2017-10-16
2017-10-17
2017-10-18
2017-10-19
2017-10-20
2017-10-21
2017-10-22
2017-10-23
2017-10-24
2017-10-25
2017-10-26
2017-10-27
2017-10-28
2017-10-29
2017-10-30
2017-10-31
2017-11-01
2017-11-02
2017-11-03
2017-11-04
2017-11-05
2017-11-06
2017-11-07
2017-11-08
2017-11-09
2017-11-10
2017-11-11
2017-11-12
2017-11-13
2017-11-14
2017-11-15
2017-11-16
2017-11-17

2019-09-03
2019-09-04
2019-09-05
2019-09-06
2019-09-07
2019-09-08
2019-09-09
2019-09-10
2019-09-11
2019-09-12
2019-09-13
2019-09-14
2019-09-15
2019-09-16
2019-09-17
2019-09-18
2019-09-19
2019-09-20
2019-09-21
2019-09-22
2019-09-23
2019-09-24
2019-09-25
2019-09-26
2019-09-27
2019-09-28
2019-09-29
2019-09-30
2019-10-01
2019-10-02
2019-10-03
2019-10-04
2019-10-05
2019-10-06
2019-10-07
2019-10-08
2019-10-09
2019-10-10
2019-10-11
2019-10-12
2019-10-13
2019-10-14
2019-10-15
2019-10-16
2019-10-17
2019-10-18
2019-10-19
2019-10-20
2019-10-21
2019-10-22
2019-10-23
2019-10-24
2019-10-25
2019-10-26
2019-10-27
2019-10-28
2019-10-29
2019-10-30
2019-10-31
2019-11-01
2019-11-02
2019-11-03
2019-11-04
2019-11-05
2019-11-06
2019-11-07
2019-11-08
2019-11-09
2019-11-10
2019-11-11
2019-11-12
2019-11-13
2019-11-14
2019-11-15
2019-11-16
2019-11-17
2019-11-18
2019-11-19
2019-11-20
2019-11-21
2019-11-22
2019-11-23
2019-11-24
2019-11-25
2019-11-26
2019-11-27
2019-11-28
2019-11-29
2019-11-30
2019-12-01
2019-12-02

2021-09-17
2021-09-18
2021-09-19
2021-09-20
2021-09-21
2021-09-22
2021-09-23
2021-09-24
2021-09-25
2021-09-26
2021-09-27
2021-09-28
2021-09-29
2021-09-30
2021-10-01
2021-10-02
2021-10-03
2021-10-04
2021-10-05
2021-10-06
2021-10-07
2021-10-08
2021-10-09
2021-10-10
2021-10-11
2021-10-12
2021-10-13
2021-10-14
2021-10-15
2021-10-16
2021-10-17
2021-10-18
2021-10-19
2021-10-20
2021-10-21
2021-10-22
2021-10-23
2021-10-24
2021-10-25
2021-10-26
2021-10-27
2021-10-28
2021-10-29
2021-10-30
2021-10-31
2021-11-01
2021-11-02
2021-11-03
2021-11-04
2021-11-05
2021-11-06
2021-11-07
2021-11-08
2021-11-09
2021-11-10
2021-11-11
2021-11-12
2021-11-13
2021-11-14
2021-11-15
2021-11-16
2021-11-17
2021-11-18
2021-11-19
2021-11-20
2021-11-21
2021-11-22
2021-11-23
2021-11-24
2021-11-25
2021-11-26
2021-11-27
2021-11-28
2021-11-29
2021-11-30
2021-12-01
2021-12-02
2021-12-03
2021-12-04
2021-12-05
2021-12-06
2021-12-07
2021-12-08
2021-12-09
2021-12-10
2021-12-11
2021-12-12
2021-12-13
2021-12-14
2021-12-15
2021-12-16

2023-10-03
2023-10-04
2023-10-05
2023-10-06
2023-10-07
2023-10-08
2023-10-09
2023-10-10
2023-10-11
2023-10-12
2023-10-13
2023-10-14
2023-10-15
2023-10-16
2023-10-17
2023-10-18
2023-10-19
2023-10-20
2023-10-21
2023-10-22
2023-10-23
2023-10-24
2023-10-25
2023-10-26
2023-10-27
2023-10-28
2023-10-29
2023-10-30
2023-10-31
2023-11-01
2023-11-02
2023-11-03
2023-11-04
2023-11-05
2023-11-06
2023-11-07
2023-11-08
2023-11-09
2023-11-10
2023-11-11
2023-11-12
2023-11-13
2023-11-14
2023-11-15
2023-11-16
2023-11-17
2023-11-18
2023-11-19
2023-11-20
2023-11-21
2023-11-22
2023-11-23
2023-11-24
2023-11-25
2023-11-26
2023-11-27
2023-11-28
2023-11-29
2023-11-30
2023-12-01
2023-12-02
2023-12-03
2023-12-04
2023-12-05
2023-12-06
2023-12-07
2023-12-08
2023-12-09
2023-12-10
2023-12-11
2023-12-12
2023-12-13
2023-12-14
2023-12-15
2023-12-16
2023-12-17
2023-12-18
2023-12-19
2023-12-20
2023-12-21
2023-12-22
2023-12-23
2023-12-24
2023-12-25
2023-12-26
2023-12-27
2023-12-28
2023-12-29
2023-12-30
2023-12-31
2024-01-01

In [5]:
#just a little experiment

#in this we assume that party positions remain the same during the next assembly term of each state
#and we try and see how alliance strength in the RS changes as a result

#TO-DO How do you make the program consider the next round of Rajya Sabha elections as well?

import csv
import datetime
import pandas as pd
import collections
import math

df_assembly_dates = pd.read_csv('assembly_end_dates.csv')

df_assembly_dates['term_ends'] = pd.to_datetime(df_assembly_dates['term_ends'], format="%d-%m-%Y").dt.date

df_assembly_data = pd.read_csv('assembly_alliances_strength.csv')

df_rs_data = pd.read_csv('rajya_sabha_alliances_strength.csv')

df_rs_data['retirement_date'] = pd.to_datetime(df_rs_data['retirement_date'], format="%d-%m-%Y").dt.date

headings = ['date', 'NDA', 'UPA', 'Others']

with open("alliances_over_time_experiment.csv", "wb") as filex:  
    wr = csv.writer(filex, delimiter = ',' , quotechar = '"' )
    wr.writerow(headings)

start = datetime.datetime.strptime("19-08-2017", "%d-%m-%Y")
end = datetime.datetime.strptime("01-07-2024", "%d-%m-%Y")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for this_date_raw in date_generated:
    this_date = this_date_raw.date()
    print this_date
    NDA = 0
    UPA = 0
    
    for index, row in df_rs_data.iterrows():

        #rs_seat_retirement_date_raw = row['retirement_date']
        #rs_seat_retirement_date = datetime.datetime.strptime(rs_seat_retirement_date_raw, "%d-%m-%Y")
        rs_seat_retirement_date = row['retirement_date']
        
        alliance = row['alliance']
        
        state = row['state']
        
        rs_mp_name = row['name'] #note this could be marked 'VACANT' too, have contingency plans for that
        
        if state != 'Nominated':
            #assembly_end_date_raw = df_assembly_dates[df_assembly_dates['state'] == state]['term_ends'].values[0]
            #assembly_end_date = datetime.datetime.strptime(assembly_end_date_raw, "%d-%m-%Y")
         #   assembly_end_date = df_assembly_dates[df_assembly_dates['state'] == state]['term_ends'].values[0]
        
            assembly_end_date = df_assembly_dates[df_assembly_dates['state'] == state]['term_ends'].values[0]
        
            #assembly_end_dates only matter for how seats marked VACANT are filled before their terms are over
            #otherwise in this experiment party positions are permanent

            #SHOULD I INDENT ALL THE LINES BELOW THIS?

            if this_date <= rs_seat_retirement_date:
                if rs_mp_name != 'VACANT':

                    if alliance == 'NDA':
                        NDA += 1
                    elif alliance == 'UPA':
                        UPA += 1

                else: #if seat is vacant

                    if this_date <= assembly_end_date: #you need to only check assembly end date for seats VACANT now

                        diff_bw_this_date_retirement_date = rs_seat_retirement_date - this_date

                        if diff_bw_this_date_retirement_date.days > 365:  #am I doubling up conditions?

                           # next_this_state_retirement_date = df_rs_data[(df_rs_data['state'] == state) &\
                                                                         #do i use 'and' here?
                            #                   (df_rs_data['retirement_date'] < rs_seat_retirement_date)].min()["retirement_date"]

                            #next_any_state_retirement_date = df_rs_data[(df_rs_data['state'] != state) &\
                                                                        #do i use 'and' here?
                             #                  (df_rs_data['retirement_date'] < rs_seat_retirement_date)].min()["retirement_date"]

                            next_any_state_retirement_date = df_rs_data[(df_rs_data['retirement_date'] >= this_date) &\
                                           (df_rs_data['retirement_date'] < rs_seat_retirement_date)].min()["retirement_date"]

                            #date_difference_this_state = rs_seat_retirement_date - next_this_state_retirement_date

                            date_difference_any_state = rs_seat_retirement_date - next_any_state_retirement_date

                            if date_difference_any_state.days > 365:

                                if this_date > next_any_state_retirement_date:

                                    seats_vacant_now_state_v2 = len(df_rs_data[(df_rs_data['state'] == state) &\
                                                      (df_rs_data['retirement_date'] == rs_seat_retirement_date) &\
                                                      (rs_mp_name == 'VACANT')])

                                    assembly_seats_total_v2 = df_assembly_data[df_assembly_data['state'] == state].sum()["seats"]

                                    droop_quota_v2 = math.floor((assembly_seats_total_v2 * 100)/(seats_vacant_now_state_v2 + 1)) + 1

                                    nda_strength_v2 = df_assembly_data[(df_assembly_data['state'] == state) &\
                                                   (df_assembly_data['alliance'] == "NDA")].sum()["seats"]

                                    upa_strength_v2 = df_assembly_data[(df_assembly_data['state'] == state) &\
                                                   (df_assembly_data['alliance'] == "UPA")].sum()["seats"]

                                    nda_seats_v2 = math.floor((nda_strength_v2 * 100)/droop_quota_v2)

                                    upa_seats_v2 = math.floor((upa_strength_v2 * 100)/droop_quota_v2)

                                    NDA += nda_seats_v2/seats_vacant_now_state_v2

                                    UPA += upa_seats_v2/seats_vacant_now_state_v2
            
            else: # ie. this_date > rs_seat_retirement_date

                if state not in ['Delhi', 'Punjab']:

                    vacancies_number_v3 = len(df_rs_data[(df_rs_data['state'] == state) &\
                                                      (df_rs_data['retirement_date'] == rs_seat_retirement_date)])

                    assembly_seats_total_v3 = df_assembly_data[df_assembly_data['state'] == state].sum()["seats"]

                    droop_quota_v3 = math.floor((assembly_seats_total_v3 * 100)/(vacancies_number_v3 + 1)) + 1

                    nda_strength_v3 = df_assembly_data[(df_assembly_data['state'] == state) &\
                                   (df_assembly_data['alliance'] == "NDA")].sum()["seats"]

                    upa_strength_v3 = df_assembly_data[(df_assembly_data['state'] == state) &\
                                   (df_assembly_data['alliance'] == "UPA")].sum()["seats"]

                    nda_seats_v3 = math.floor((nda_strength_v3 * 100)/droop_quota_v3)

                    upa_seats_v3 = math.floor((upa_strength_v3 * 100)/droop_quota_v3)

                    NDA += nda_seats_v3/vacancies_number_v3

                    UPA += upa_seats_v3/vacancies_number_v3

                elif state == 'Delhi':

                    vacancies_number_d = 1 #or should it be 3, how do you deal with sep. elections issue?

                    assembly_seats_total_d = df_assembly_data[df_assembly_data['state'] == 'Delhi'].sum()["seats"]

                    droop_quota_d = math.floor((assembly_seats_total_d * 100)/(vacancies_number_d + 1)) + 1

                    nda_strength_d = df_assembly_data[(df_assembly_data['state'] == 'Delhi') &\
                                   (df_assembly_data['alliance'] == "NDA")].sum()["seats"]

                    upa_strength_d = df_assembly_data[(df_assembly_data['state'] == 'Delhi') &\
                                   (df_assembly_data['alliance'] == "UPA")].sum()["seats"]

                    nda_seats_d = math.floor((nda_strength_d * 100)/droop_quota_d)

                    upa_seats_d = math.floor((upa_strength_d * 100)/droop_quota_d)

                    NDA += nda_seats_d #Do I need to divide by vacancies_number_d here?

                    UPA += upa_seats_d 

                elif state == 'Punjab':

                    punjab_dates_list = df_rs_data[df_rs_data['state'] == 'Punjab']['retirement_date'].tolist()

                    punjab_dates_freq = dict(collections.Counter(punjab_dates_list))

                    if punjab_dates_freq[rs_seat_retirement_date] == 5: #or should I do '5'?

                        vacancies_number_p_g2 = 2

                        vacancies_number_p_g3 = 3

                        assembly_seats_total_p = df_assembly_data[df_assembly_data['state'] == 'Punjab'].sum()["seats"]                    

                        droop_quota_p_g2 = math.floor((assembly_seats_total_p * 100)/(vacancies_number_p_g2 + 1)) + 1

                        droop_quota_p_g3 = math.floor((assembly_seats_total_p * 100)/(vacancies_number_p_g3 + 1)) + 1

                        nda_strength_p = df_assembly_data[(df_assembly_data['state'] == 'Punjab') &\
                                       (df_assembly_data['alliance'] == "NDA")].sum()["seats"]

                        upa_strength_p = df_assembly_data[(df_assembly_data['state'] == 'Punjab') &\
                                       (df_assembly_data['alliance'] == "UPA")].sum()["seats"]

                        nda_seats_p_g2 = math.floor((nda_strength_p * 100)/droop_quota_p_g2)

                        upa_seats_p_g2 = math.floor((upa_strength_p * 100)/droop_quota_p_g2)

                        nda_seats_p_g3 = math.floor((nda_strength_p * 100)/droop_quota_p_g3)

                        upa_seats_p_g3 = math.floor((upa_strength_p * 100)/droop_quota_p_g3)

                        NDA += (nda_seats_p_g2 + nda_seats_p_g3)/5 #Is dividing by 5 the right thing here?

                        UPA += (upa_seats_p_g2 + upa_seats_p_g3)/5 


                    else:

                        vacancies_number_p2 = len(df_rs_data[(df_rs_data['state'] == 'Punjab') &\
                                      (df_rs_data['retirement_date'] == rs_seat_retirement_date)])

                        assembly_seats_total_p = df_assembly_data[df_assembly_data['state'] == 'Punjab'].sum()["seats"]

                        droop_quota_p2 = math.floor((assembly_seats_total_p * 100)/(vacancies_number_p2 + 1)) + 1

                        nda_strength_p = df_assembly_data[(df_assembly_data['state'] == 'Punjab') &\
                                       (df_assembly_data['alliance'] == "NDA")].sum()["seats"]

                        upa_strength_p = df_assembly_data[(df_assembly_data['state'] == 'Punjab') &\
                                       (df_assembly_data['alliance'] == "UPA")].sum()["seats"]

                        nda_seats_p2 = math.floor((nda_strength_p * 100)/droop_quota_p2)

                        upa_seats_p2 = math.floor((upa_strength_p * 100)/droop_quota_p2)

                        NDA += nda_seats_p2/vacancies_number_p2 #Do I need to divide by vacancies_number_p here?

                        UPA += upa_seats_p2/vacancies_number_p2 
        
        else: #if MP is nominated
            if this_date <= rs_seat_retirement_date:
                if alliance == 'NDA':
                    NDA += 1
                elif alliance == 'UPA':
                    UPA += 1
            #no else condition here, if after retirement date, we can't say anything about who will get nominated
            #and if they are otherwise politically affiliated or not

    OTHERS = 245 - (NDA + UPA)
    
    csv_row = list()
    csv_row.append(this_date)
    csv_row.append(NDA)
    csv_row.append(UPA)
    csv_row.append(OTHERS)
     
    with open("alliances_over_time_experiment.csv", "ab") as filez:
        wrz = csv.writer(filez, delimiter = ',' , quotechar = '"' )
        wrz.writerow(csv_row)

filez.close()

print "program done"

2017-08-19
2017-08-20
2017-08-21
2017-08-22
2017-08-23
2017-08-24
2017-08-25
2017-08-26
2017-08-27
2017-08-28
2017-08-29
2017-08-30
2017-08-31
2017-09-01
2017-09-02
2017-09-03
2017-09-04
2017-09-05
2017-09-06
2017-09-07
2017-09-08
2017-09-09
2017-09-10
2017-09-11
2017-09-12
2017-09-13
2017-09-14
2017-09-15
2017-09-16
2017-09-17
2017-09-18
2017-09-19
2017-09-20
2017-09-21
2017-09-22
2017-09-23
2017-09-24
2017-09-25
2017-09-26
2017-09-27
2017-09-28
2017-09-29
2017-09-30
2017-10-01
2017-10-02
2017-10-03
2017-10-04
2017-10-05
2017-10-06
2017-10-07
2017-10-08
2017-10-09
2017-10-10
2017-10-11
2017-10-12
2017-10-13
2017-10-14
2017-10-15
2017-10-16
2017-10-17
2017-10-18
2017-10-19
2017-10-20
2017-10-21
2017-10-22
2017-10-23
2017-10-24
2017-10-25
2017-10-26
2017-10-27
2017-10-28
2017-10-29
2017-10-30
2017-10-31
2017-11-01
2017-11-02
2017-11-03
2017-11-04
2017-11-05
2017-11-06
2017-11-07
2017-11-08
2017-11-09
2017-11-10
2017-11-11
2017-11-12
2017-11-13
2017-11-14
2017-11-15
2017-11-16
2017-11-17

2019-09-03
2019-09-04
2019-09-05
2019-09-06
2019-09-07
2019-09-08
2019-09-09
2019-09-10
2019-09-11
2019-09-12
2019-09-13
2019-09-14
2019-09-15
2019-09-16
2019-09-17
2019-09-18
2019-09-19
2019-09-20
2019-09-21
2019-09-22
2019-09-23
2019-09-24
2019-09-25
2019-09-26
2019-09-27
2019-09-28
2019-09-29
2019-09-30
2019-10-01
2019-10-02
2019-10-03
2019-10-04
2019-10-05
2019-10-06
2019-10-07
2019-10-08
2019-10-09
2019-10-10
2019-10-11
2019-10-12
2019-10-13
2019-10-14
2019-10-15
2019-10-16
2019-10-17
2019-10-18
2019-10-19
2019-10-20
2019-10-21
2019-10-22
2019-10-23
2019-10-24
2019-10-25
2019-10-26
2019-10-27
2019-10-28
2019-10-29
2019-10-30
2019-10-31
2019-11-01
2019-11-02
2019-11-03
2019-11-04
2019-11-05
2019-11-06
2019-11-07
2019-11-08
2019-11-09
2019-11-10
2019-11-11
2019-11-12
2019-11-13
2019-11-14
2019-11-15
2019-11-16
2019-11-17
2019-11-18
2019-11-19
2019-11-20
2019-11-21
2019-11-22
2019-11-23
2019-11-24
2019-11-25
2019-11-26
2019-11-27
2019-11-28
2019-11-29
2019-11-30
2019-12-01
2019-12-02

2021-09-17
2021-09-18
2021-09-19
2021-09-20
2021-09-21
2021-09-22
2021-09-23
2021-09-24
2021-09-25
2021-09-26
2021-09-27
2021-09-28
2021-09-29
2021-09-30
2021-10-01
2021-10-02
2021-10-03
2021-10-04
2021-10-05
2021-10-06
2021-10-07
2021-10-08
2021-10-09
2021-10-10
2021-10-11
2021-10-12
2021-10-13
2021-10-14
2021-10-15
2021-10-16
2021-10-17
2021-10-18
2021-10-19
2021-10-20
2021-10-21
2021-10-22
2021-10-23
2021-10-24
2021-10-25
2021-10-26
2021-10-27
2021-10-28
2021-10-29
2021-10-30
2021-10-31
2021-11-01
2021-11-02
2021-11-03
2021-11-04
2021-11-05
2021-11-06
2021-11-07
2021-11-08
2021-11-09
2021-11-10
2021-11-11
2021-11-12
2021-11-13
2021-11-14
2021-11-15
2021-11-16
2021-11-17
2021-11-18
2021-11-19
2021-11-20
2021-11-21
2021-11-22
2021-11-23
2021-11-24
2021-11-25
2021-11-26
2021-11-27
2021-11-28
2021-11-29
2021-11-30
2021-12-01
2021-12-02
2021-12-03
2021-12-04
2021-12-05
2021-12-06
2021-12-07
2021-12-08
2021-12-09
2021-12-10
2021-12-11
2021-12-12
2021-12-13
2021-12-14
2021-12-15
2021-12-16

2023-10-02
2023-10-03
2023-10-04
2023-10-05
2023-10-06
2023-10-07
2023-10-08
2023-10-09
2023-10-10
2023-10-11
2023-10-12
2023-10-13
2023-10-14
2023-10-15
2023-10-16
2023-10-17
2023-10-18
2023-10-19
2023-10-20
2023-10-21
2023-10-22
2023-10-23
2023-10-24
2023-10-25
2023-10-26
2023-10-27
2023-10-28
2023-10-29
2023-10-30
2023-10-31
2023-11-01
2023-11-02
2023-11-03
2023-11-04
2023-11-05
2023-11-06
2023-11-07
2023-11-08
2023-11-09
2023-11-10
2023-11-11
2023-11-12
2023-11-13
2023-11-14
2023-11-15
2023-11-16
2023-11-17
2023-11-18
2023-11-19
2023-11-20
2023-11-21
2023-11-22
2023-11-23
2023-11-24
2023-11-25
2023-11-26
2023-11-27
2023-11-28
2023-11-29
2023-11-30
2023-12-01
2023-12-02
2023-12-03
2023-12-04
2023-12-05
2023-12-06
2023-12-07
2023-12-08
2023-12-09
2023-12-10
2023-12-11
2023-12-12
2023-12-13
2023-12-14
2023-12-15
2023-12-16
2023-12-17
2023-12-18
2023-12-19
2023-12-20
2023-12-21
2023-12-22
2023-12-23
2023-12-24
2023-12-25
2023-12-26
2023-12-27
2023-12-28
2023-12-29
2023-12-30
2023-12-31

In [6]:
df_real = pd.read_csv('alliances_over_time_realistic.csv')
df_exp = pd.read_csv('alliances_over_time_experiment.csv')

df_real.head()

,date,NDA,UPA,Others
0,2017-08-19,81,62,102
1,2017-08-20,81,62,102
2,2017-08-21,81,62,102
3,2017-08-22,81,62,102
4,2017-08-23,81,62,102


In [7]:
df_exp.head()

,date,NDA,UPA,Others
0,2017-08-19,81,62,102
1,2017-08-20,81,62,102
2,2017-08-21,81,62,102
3,2017-08-22,81,62,102
4,2017-08-23,81,62,102


In [8]:
#change all column names to lower case, seems d3 doesn't deal well with column names that are all upper case or
        #has underscores in them or is more than 4 letters long. Since when?? I must be doing sthing wrong elsewhere

df_real = df_real.rename(columns={'NDA': 'nda', 'UPA': 'upa', 'Others': 'oth' })
df_real.head()

,date,nda,upa,oth
0,2017-08-19,81,62,102
1,2017-08-20,81,62,102
2,2017-08-21,81,62,102
3,2017-08-22,81,62,102
4,2017-08-23,81,62,102


In [9]:
df_exp = df_exp.rename(columns={'NDA': 'ndax', 'UPA': 'upax', 'Others': 'othx' })
df_exp.head()

,date,ndax,upax,othx
0,2017-08-19,81,62,102
1,2017-08-20,81,62,102
2,2017-08-21,81,62,102
3,2017-08-22,81,62,102
4,2017-08-23,81,62,102


In [10]:
combined = pd.merge(df_real, df_exp, on=['date'])
combined.head()

,date,nda,upa,oth,ndax,upax,othx
0,2017-08-19,81,62,102,81,62,102
1,2017-08-20,81,62,102,81,62,102
2,2017-08-21,81,62,102,81,62,102
3,2017-08-22,81,62,102,81,62,102
4,2017-08-23,81,62,102,81,62,102


In [11]:
combined.to_csv('alliances_strength_combined.csv', index=False)

In [30]:
#data check

import csv
import datetime
import pandas as pd
import collections
import math

df_assembly_dates = pd.read_csv('assembly_end_dates.csv')

df_assembly_dates['term_ends'] = pd.to_datetime(df_assembly_dates['term_ends'], format="%d-%m-%Y").dt.date
        
df_assembly_data = pd.read_csv('assembly_alliances_strength.csv')

df_rs_data = pd.read_csv('rajya_sabha_alliances_strength.csv')

df_rs_data['retirement_date'] = pd.to_datetime(df_rs_data['retirement_date'], format="%d-%m-%Y").dt.date


headings = ['date', 'NDA', 'UPA', 'Others']

start = datetime.datetime.strptime("04-04-2018", "%d-%m-%Y")
end = datetime.datetime.strptime("05-04-2018", "%d-%m-%Y")
date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for this_date_raw in date_generated:
    this_date = this_date_raw.date()
    print this_date
    NDA = 0
    UPA = 0
    NOM = 0
    seats_post_retirement = 0
    seats_post_retirement_before_assembly_end = 0
    
    for index, row in df_rs_data.iterrows():
        rs_seat_retirement_date = row['retirement_date']

        alliance = row['alliance']
        
        state = row['state']
        
        rs_mp_name = row['name'] #note this could be marked 'VACANT' too, have contingency plans for that
        
        if state != 'Nominated':
            

            assembly_end_date = df_assembly_dates[df_assembly_dates['state'] == state]['term_ends'].values[0]
        
#             if this_date <= rs_seat_retirement_date: 
#                 if rs_mp_name != 'VACANT':

#                     if alliance == 'NDA':
#                         NDA += 1
#                     elif alliance == 'UPA':
#                         UPA += 1

#                 else:  # ie. if seat is marked 'vacant'

#                     if this_date <= assembly_end_date:

#                         diff_bw_this_date_retirement_date = rs_seat_retirement_date - this_date

#                         if diff_bw_this_date_retirement_date.days > 365:

#                             next_any_state_retirement_date = df_rs_data[(df_rs_data['retirement_date'] >= this_date) &\
#                                                (df_rs_data['retirement_date'] < rs_seat_retirement_date)].min()["retirement_date"]

#                             date_difference_any_state = rs_seat_retirement_date - next_any_state_retirement_date

#                             if date_difference_any_state.days > 365:

#                                 if this_date > next_any_state_retirement_date:

#                                     seats_vacant_now_state2 = len(df_rs_data[(df_rs_data['state'] == state) &\
#                                                       (df_rs_data['retirement_date'] == rs_seat_retirement_date) &\
#                                                       (rs_mp_name == 'VACANT')])

#                                     assembly_seats_total_v2 = df_assembly_data[df_assembly_data['state'] == state].sum()["seats"]

#                                     droop_quota_v2 = math.floor((assembly_seats_total_v2 * 100)/(seats_vacant_now_state2 + 1)) + 1

#                                     nda_strength_v2 = df_assembly_data[(df_assembly_data['state'] == state) &\
#                                                    (df_assembly_data['alliance'] == "NDA")].sum()["seats"]
#                                     #code above from https://stackoverflow.com/a/20995313

#                                     upa_strength_v2 = df_assembly_data[(df_assembly_data['state'] == state) &\
#                                                    (df_assembly_data['alliance'] == "UPA")].sum()["seats"]

#                                     nda_seats_v2 = math.floor((nda_strength_v2 * 100)/droop_quota_v2)

#                                     upa_seats_v2 = math.floor((upa_strength_v2 * 100)/droop_quota_v2)

#                                     NDA += nda_seats_v2/seats_vacant_now_state2

#                                     UPA += upa_seats_v2/seats_vacant_now_state2

#             else: # ie. this_date > rs_seat_retirement_date

            if this_date > rs_seat_retirement_date:
                
                seats_post_retirement += 1
            
#                 if this_date <= assembly_end_date:
    #                    if state =='Uttar Pradesh':

                seats_post_retirement_before_assembly_end += 1

#                     if state not in ['Delhi', 'Punjab']:

                vacancies_number_v3 = len(df_rs_data[(df_rs_data['state'] == state) &\
                                                  (df_rs_data['retirement_date'] == rs_seat_retirement_date)])

                #print "there are {} vacancies.".format(vacancies_number_v3)

                assembly_seats_total_v3 = df_assembly_data[df_assembly_data['state'] == state].sum()["seats"]

                #print "there are {} assembly seats.".format(assembly_seats_total_v3)

                droop_quota_v3 = math.floor((assembly_seats_total_v3 * 100)/(vacancies_number_v3 + 1)) + 1

                #print "the droop quota is {}.".format(droop_quota_v3)

                nda_strength_v3 = df_assembly_data[(df_assembly_data['state'] == state) &\
                               (df_assembly_data['alliance'] == "NDA")].sum()["seats"]
                #code above from https://stackoverflow.com/a/20995313

                #print "the nda_strength is {}.".format(nda_strength_v3)

                upa_strength_v3 = df_assembly_data[(df_assembly_data['state'] == state) &\
                               (df_assembly_data['alliance'] == "UPA")].sum()["seats"]

                #print "the upa_strength is {}.".format(upa_strength_v3)

                nda_seats_v3 = math.floor((nda_strength_v3 * 100)/droop_quota_v3)

                #print "the nda seats are {}.".format(nda_seats_v3)

                upa_seats_v3 = math.floor((upa_strength_v3 * 100)/droop_quota_v3)

                #print "the upa seats are {}.".format(upa_seats_v3)

                NDA += nda_seats_v3/vacancies_number_v3

                #print "what will get added to NDA variable is {}.".format(NDA)

                UPA += upa_seats_v3/vacancies_number_v3

                    #print "what will get added to UPA variable is {}.".format(UPA)

#                     elif state == 'Delhi':

#                         vacancies_number_d = 1 

#                         assembly_seats_total_d = df_assembly_data[df_assembly_data['state'] == 'Delhi'].sum()["seats"]

#                         droop_quota_d = math.floor((assembly_seats_total_d * 100)/(vacancies_number_d + 1)) + 1

#                         nda_strength_d = df_assembly_data[(df_assembly_data['state'] == 'Delhi') &\
#                                        (df_assembly_data['alliance'] == "NDA")].sum()["seats"]
#                         #code above from https://stackoverflow.com/a/20995313

#                         upa_strength_d = df_assembly_data[(df_assembly_data['state'] == 'Delhi') &\
#                                        (df_assembly_data['alliance'] == "UPA")].sum()["seats"]

#                         nda_seats_d = math.floor((nda_strength_d * 100)/droop_quota_d)

#                         upa_seats_d = math.floor((upa_strength_d * 100)/droop_quota_d)

#                         NDA += nda_seats_d #Do I need to divide by vacancies_number_d here? No

#                         UPA += upa_seats_d 

#                     elif state == 'Punjab':

#                         punjab_dates_list = df_rs_data[df_rs_data['state'] == 'Punjab']['retirement_date'].tolist()

#                         punjab_dates_freq = dict(collections.Counter(punjab_dates_list))

#                         if punjab_dates_freq[rs_seat_retirement_date] == 5: #if the retirement date is the one for the five seats

#                             vacancies_number_p_g2 = 2 #g2 for group of two

#                             vacancies_number_p_g3 = 3 #g3 for group of three

#                             assembly_seats_total_p = df_assembly_data[df_assembly_data['state'] == 'Punjab'].sum()["seats"]                    

#                             droop_quota_p_g2 = math.floor((assembly_seats_total_p * 100)/(vacancies_number_p_g2 + 1)) + 1

#                             droop_quota_p_g3 = math.floor((assembly_seats_total_p * 100)/(vacancies_number_p_g3 + 1)) + 1

#                             nda_strength_p = df_assembly_data[(df_assembly_data['state'] == 'Punjab') &\
#                                            (df_assembly_data['alliance'] == "NDA")].sum()["seats"]
#                             #code above from https://stackoverflow.com/a/20995313

#                             upa_strength_p = df_assembly_data[(df_assembly_data['state'] == 'Punjab') &\
#                                            (df_assembly_data['alliance'] == "UPA")].sum()["seats"]

#                             nda_seats_p_g2 = math.floor((nda_strength_p * 100)/droop_quota_p_g2)

#                             upa_seats_p_g2 = math.floor((upa_strength_p * 100)/droop_quota_p_g2)

#                             nda_seats_p_g3 = math.floor((nda_strength_p * 100)/droop_quota_p_g3)

#                             upa_seats_p_g3 = math.floor((upa_strength_p * 100)/droop_quota_p_g3)

#                             NDA += (nda_seats_p_g2 + nda_seats_p_g3)/5  #Is dividing by 5 the right thing here?

#                             UPA += (upa_seats_p_g2 + upa_seats_p_g3)/5 


#                         else: #if it's the group of two RS seats from Punjab that share the same retirement date

#                             vacancies_number_p2 = len(df_rs_data[(df_rs_data['state'] == 'Punjab') &\
#                                           (df_rs_data['retirement_date'] == rs_seat_retirement_date)])

#                             assembly_seats_total_p2 = df_assembly_data[df_assembly_data['state'] == 'Punjab'].sum()["seats"]

#                             droop_quota_p2 = math.floor((assembly_seats_total_p2 * 100)/(vacancies_number_p2 + 1)) + 1

#                             nda_strength_p2 = df_assembly_data[(df_assembly_data['state'] == 'Punjab') &\
#                                            (df_assembly_data['alliance'] == "NDA")].sum()["seats"]
#                             #code above from https://stackoverflow.com/a/20995313

#                             upa_strength_p2 = df_assembly_data[(df_assembly_data['state'] == 'Punjab') &\
#                                            (df_assembly_data['alliance'] == "UPA")].sum()["seats"]

#                             #below I'm calculating the number of seats any party will get in an RS election for a 
#                                 #specific number of vacancies

#                             nda_seats_p2 = math.floor((nda_strength_p2 * 100)/droop_quota_p2)

#                             upa_seats_p2 = math.floor((upa_strength_p2 * 100)/droop_quota_p2)

#                             NDA += nda_seats_p2/vacancies_number_p2 #Do I need to divide by vacancies_number_p here?

#                             UPA += upa_seats_p2/vacancies_number_p2 

#         else: #if it is a nominated seat, the 'state' is marked 'nominated' in the data
#             if this_date <= rs_seat_retirement_date:
#                 if alliance == 'NDA':
#                     NDA += 1
#                 elif alliance == 'UPA':
#                     UPA += 1
#                 #and if there is no alliance affiliation recorded against that MP, do nothing
#                 #and if the date is after the retirement date, we can't say anything anyway
#             if this_date > rs_seat_retirement_date:
#                 seats_post_retirement += 1
#                 NOM += 1

    print "The no. of seats added for NDA is {}.".format(NDA)
    print "The no. of seats added for UPA is {}.".format(UPA)
    #print NOM
    print "The no. of vacancies is {}.".format(seats_post_retirement)
    print "The no. of vacancies before assembly end is {}.".format(seats_post_retirement_before_assembly_end)
    #OTHERS = 245 - (NDA + UPA)
    #print OTHERS
    
#    csv_row = list()
#    csv_row.append(this_date)
#    csv_row.append(NDA)
#    csv_row.append(UPA)
#    csv_row.append(OTHERS)
     
 #   with open("alliances_over_time_realistic.csv", "ab") as filez:
  #      wrz = csv.writer(filez, delimiter = ',' , quotechar = '"' )
   #     wrz.writerow(csv_row)

#filez.close()

print "program done"

2018-04-04
The no. of seats added for NDA is 31.0.
The no. of seats added for UPA is 6.0.
The no. of vacancies is 60.
The no. of vacancies before assembly end is 60.
program done
